In [30]:
import imaplib, email, os, json
from email.header import decode_header

from dotenv import load_dotenv
from langchain import LLMChain
from langchain_openai import ChatOpenAI
from langchain.prompts import PromptTemplate
load_dotenv()

True

login

In [31]:
IMAP_SERVER = 'imap.gmx.com'
IMAP_PORT = 993
EMAIL_ACCOUNT = os.environ['EMAIL']
EMAIL_PASSWORD = os.environ['EMAIL_PASSWORD']
mail = imaplib.IMAP4_SSL(IMAP_SERVER, IMAP_PORT)
mail.login(EMAIL_ACCOUNT, EMAIL_PASSWORD)

('OK', [b'LOGIN completed'])

In [32]:
mail.select('INBOX')
status, messages = mail.search(None, 'ALL')
email_ids = messages[0].split()
email_ids = [int(eid) for eid in email_ids]

In [33]:
def get_email_body(msg):
    body = ""
    if msg.is_multipart():
        # If multipart, iterate through the parts
        for part in msg.walk():
            # Check if the content type is text/plain or text/html
            content_type = part.get_content_type()
            content_disposition = str(part.get("Content-Disposition"))

            if content_type == "text/plain" and "attachment" not in content_disposition:
                # Extract plain text
                body = part.get_payload(decode=True).decode(part.get_content_charset() or "utf-8")
                break
            elif content_type == "text/html" and "attachment" not in content_disposition:
                # Extract HTML
                body = part.get_payload(decode=True).decode(part.get_content_charset() or "utf-8")
    else:
        # If not multipart, check the content type
        content_type = msg.get_content_type()
        if content_type == "text/plain" or content_type == "text/html":
            body = msg.get_payload(decode=True).decode(msg.get_content_charset() or "utf-8")
    return str(body)

In [34]:
llm = ChatOpenAI(model = 'gpt-4o-mini', temperature=0)

classification_prompt = PromptTemplate.from_template(
    '''
        Task:
        You are my (florian kenner's) personal assistant who is instructed to cleanup his mailbox. Rate the following email, whether to keep or delete it. You want to generally delete all emails which will not be of any importance in the future, such as ads, spam, newsletters outdated information, such as one time authentication codes etc.
        Keep all emails which are personal communications, bookings, invoices, important notifications etc.
        If you are not totally sure, keep the email.

        Email Title: {header}
        Email Sender: {sender}
        Date: {date}
        Email Content: {content}


           Format of Expected Output: Please provide your classification and justification in the following structured format:
            {{
                "classification": "KEEP" or "DELETE",
                "justification": "A one short sentence explanation of why you made this classification based on the provided email"
            }}
    '''
)

In [35]:
BACKUP_FOLDER = "./email_backups"
import os
import re
import json

BACKUP_FOLDER = "./email_backups"

def save_email_backup(email_id, message, metadata):
    # Create backup folder if it doesn't exist
    os.makedirs(BACKUP_FOLDER, exist_ok=True)

    # Sanitize and shorten subject
    subject = ''.join(c for c in metadata["subject"][:50] if c.isalnum())
    subject = re.sub(r'[<>:"/\\|?*\r\n]+', '_', subject)

    # Save raw email content as .eml
    filename = f"{BACKUP_FOLDER}/{email_id}_{subject}.eml"
    with open(filename, 'wb') as out:  # Binary mode for raw email
        out.write(message.as_bytes())  # Save raw bytes of email

    # Encode the body to handle special characters
    metadata["body"] = re.sub(r'[^\x00-\x7F]+', '', metadata["body"]).encode('utf-8', 'replace').decode('utf-8')
    # Save metadata as .json
    json_filename = f"{BACKUP_FOLDER}/{email_id}_{subject}.json"
    with open(json_filename, "w", encoding = 'utf-8') as f:
        json.dump(metadata, f, ensure_ascii=False)

    save_attachments(message)

def save_attachments(msg, folder="./attachments"):
    os.makedirs(folder, exist_ok=True)

    for part in msg.walk():
        if part.get_content_disposition() == "attachment":
            filename = part.get_filename()
            filename = re.sub(r'[<>:"/\\|?*\r\n]+', '_', filename)
            if filename:
                filepath = os.path.join(folder, filename)
                with open(filepath, "wb") as f:
                    f.write(part.get_payload(decode=True))
    

classify and delete

In [ ]:
from tqdm import tqdm

deleted = 0
kept = 0
checked = set()

if os.path.exists('./checked.txt'):
    with open('./checked.txt') as f:
        content = f.read()
        checked = set(json.loads(content))

for id in tqdm(email_ids):
    if id in checked: 
        continue

    _, msg_data = mail.fetch(str(id), '(RFC822)')
    message = email.message_from_bytes(msg_data[0][1])
    subject, encoding = decode_header(message['subject'])[0]
    if isinstance(subject, bytes):
        subject = subject.decode(encoding if encoding else "utf-8")

    sender = message.get('FROM')
    date = message.get('Date')
    body = get_email_body(message)

    

    # Save the email to the folder email_backup
    save_email_backup(id, message, {
    "email_id": id,
    "subject": str(subject),
    "sender": str(sender),
    "date": str(date),
    "body": body
})


    agent = LLMChain(llm = llm, prompt = classification_prompt)

    result_raw = agent.invoke({
        'header': subject,
        'sender': sender,
        'date': date,
        'content': body
    })
    result_str = result_raw.get('text', '')
    result = json.loads(result_str)
    
    if result['classification'] == 'DELETE':
        mail.store(str(id), '+FLAGS', '\\Deleted')

        mail.expunge()
        deleted += 1
    else:
        kept += 1

    checked.add(id)

    with open('checked.txt', 'w') as f:
        f.write(json.dumps(list(checked)))

  2%|▏         | 103/5572 [00:30<1:29:36,  1.02it/s]

In [ ]:
metadata = {'email_id': 54, 'subject': 'Ihr Steam-Account: Verifizierung der E-Mail-Adresse', 'sender': '<email.header.Header object at 0x00000244BA6266E0>', 'date': 'Sun, 24 Sep 2017 07:14:48 -0700', 'body': '\r\nHallo flo_2k03,\r\n\r\n\r\nUm die Sicherheit Ihres Steam-Accounts zu gewhrleisten, verifizieren Sie bitte Ihre E-Mail-Adresse.\r\n<a href="https://steamcommunity.com/actions/validateemail?stoken=388081856_17137655_BBB8C1E55314561BA89241B42259058D04A1E462">Klicken Sie hier, um Ihre E-Mail-Adresse zu verifizieren.</a>\r\n\r\nWenn Sie Ihre E-Mail-Adresse verifizieren, knnen Sie: Sicherheit durch Steam Guard nutzen, Ihre Steam-Anmeldedaten ndern, Handels- und Marktbesttigungen benutzen und den Zugriff auf Ihren Steam-Account wiedererlangen, wenn Sie ihn verlieren oder Ihr Passwort vergessen sollten.\r\n\r\nVielen Dank, dass Sie uns dabei helfen, die Sicherheit Ihres Accounts sicherzustellen.\r\n\r\n\r\nDas Team von Steam\r\nhttps://help.steampowered.com\r\n\r\n\r\n\r\n==============\r\nDiese Benachrichtigung wurde an die mit Ihrem Steam-Account verknpfte E-Mail-Adresse gesendet.\r\nWeitere Informationen zu den Datenschutzrichtlinien von Valve finden Sie unter: http://www.valvesoftware.com/privacy.htm\r\nBitte antworten Sie nicht auf diese automatisch generierte E-Mail.\r\n\r\n Valve Corporation. Alle Rechte vorbehalten. Alle Marken sind Eigentum ihrer jeweiligen Besitzer in den USA und anderen Lndern.\r\n\r\n'}

In [ ]:
with open('./a.json', 'w') as f:
    json.dump(metadata, f)